# Qiskit Primitives

The existing Qiskit interface to backends (backend.run()) was originally designed to accept a list of circuits and return counts for every job. Over time, it became clear that users have diverse purposes for quantum computing, and therefore the ways in which they define the requirements for their computing jobs are expanding. 

To address this issue, Qiskit introduced the `qiskit.primitives` module in version 0.35.0 (Qiskit Terra 0.20.0) to provide a simplified way for end users to compute outputs of interest from a QuantumCircuit and to use backends. Qiskit's primitives provide methods that make it easier to build modular algorithms and other higher-order programs. Rather than simply returning counts, they return more immediately meaningful information.

The two main submodules of the primitives are the `Sampler` and the `Estimator`. The first one computes quasi-probability distributions from circuit measurements, while the second one calculates and interprets expectation values of quantum operators that are required for many near-term quantum algorithms.

MQT now provides its own version of the Qiskit primitives:

- `Sampler` leverages the default circuit simulator based on decision diagrams, while preserving the methods and functionality of the original Qiskit's sampler.

- `Estimator` is currently in development and will be available soon.


## Sampler

The `Sampler` takes a list of `QuantumCircuit` objects and simulates them using the `QasmSimulatorBackend` from MQT DDSIM. It then computes the quasi-probability distributions of the circuits in the list and encapsulates the results, along with the job's metadata, within a `SamplerResult` object. 

Furthermore, it also handles transpilation and parameter binding when working with parametrized circuits.

Here we show an example on how to use this submodule:

In [9]:
import numpy as np
from qiskit import *

from mqt.ddsim.sampler import Sampler

# Circuit to create a Bell state
circ = QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(0, 2)
circ.measure_all()

# Show circuit
print(circ.draw(fold=-1))

# Initialize sampler
sampler = Sampler()

# Submit job
job = sampler.run(circ)
result = job.result()

        ┌───┐           ░ ┌─┐      
   q_0: ┤ H ├──■────■───░─┤M├──────
        └───┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ─────┤ X ├──┼───░──╫─┤M├───
             └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ──────────┤ X ├─░──╫──╫─┤M├
                  └───┘ ░  ║  ║ └╥┘
meas: 3/═══════════════════╩══╩══╩═
                           0  1  2 


The `result()` method of the job returns a `SamplerResult` object, which includes both the quasi-probability distribution and job metadata.

In [10]:
print(f">>> {result}")
print(f"  > Quasi-probability distribution: {result.quasi_dists[0]}")

>>> SamplerResult(quasi_dists=[{0: 0.494140625, 7: 0.505859375}], metadata=[{'shots': 1024}])
  > Quasi-probability distribution: {0: 0.494140625, 7: 0.505859375}


You can also have a list with multiple quantum circuits as an input:

In [8]:
# Create second circuit
circ2 = QuantumCircuit(3)
for qubit in circ2.qubits:
    circ2.h(qubit)
circ2.measure_all()

circuits = [circ, circ2]

for cir in circuits:
    print(cir.draw(fold=-1))

sampler = Sampler()
job = sampler.run(circuits)
result = job.result()

print(f">>> Quasi-probability distribution: {result.quasi_dists}")

        ┌───┐           ░ ┌─┐      
   q_0: ┤ H ├──■────■───░─┤M├──────
        └───┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ─────┤ X ├──┼───░──╫─┤M├───
             └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ──────────┤ X ├─░──╫──╫─┤M├
                  └───┘ ░  ║  ║ └╥┘
meas: 3/═══════════════════╩══╩══╩═
                           0  1  2 
        ┌───┐ ░ ┌─┐      
   q_0: ┤ H ├─░─┤M├──────
        ├───┤ ░ └╥┘┌─┐   
   q_1: ┤ H ├─░──╫─┤M├───
        ├───┤ ░  ║ └╥┘┌─┐
   q_2: ┤ H ├─░──╫──╫─┤M├
        └───┘ ░  ║  ║ └╥┘
meas: 3/═════════╩══╩══╩═
                 0  1  2 
>>> Quasi-probability distribution: [{0: 0.494140625, 7: 0.505859375}, {0: 0.1083984375, 1: 0.1162109375, 2: 0.138671875, 3: 0.1279296875, 4: 0.11328125, 5: 0.1240234375, 6: 0.126953125, 7: 0.14453125}]


Or parametrized circuits:

In [7]:
from qiskit.circuit import Parameter

theta_a = Parameter("theta_a")
theta_b = Parameter("theta_b")
param_circ = QuantumCircuit(1)
param_circ.rx(theta_a, 0)
param_circ.rz(theta_b, 0)
param_circ.rx(theta_a, 0)
param_circ.barrier()
param_circ.measure_all()

print(param_circ.draw(fold=-1))

sampler = Sampler()
job = sampler.run([param_circ], [[np.pi / 2, np.pi]])
result = job.result()

print(f">>> Quasi-probability distribution: {result.quasi_dists}")

        ┌─────────────┐┌─────────────┐┌─────────────┐ ░  ░ ┌─┐
     q: ┤ Rx(theta_a) ├┤ Rz(theta_b) ├┤ Rx(theta_a) ├─░──░─┤M├
        └─────────────┘└─────────────┘└─────────────┘ ░  ░ └╥┘
meas: 1/════════════════════════════════════════════════════╩═
                                                            0 
>>> Quasi-probability distribution: [{0: 1.0}]
